In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.precision', 4)

print("=" * 70)
print("SRK/T2 IOL FORMULA IMPLEMENTATION")
print("=" * 70)

print("✓ Libraries loaded successfully")

# Load data
df = pd.read_excel('FacoDMEK.xlsx', sheet_name='Data')

print(f"Number of patients: {len(df)}")
print()

# Calculate average K
df['K_avg'] = (df['Bio-Ks'] + df['Bio-Kf']) / 2

SRK/T2 IOL FORMULA IMPLEMENTATION
✓ Libraries loaded successfully
Number of patients: 96



In [2]:
def calculate_SRKT2(AL, K_avg, IOL_power, A_constant, nc=1.333, k_index=1.3375):
    """
    SRK/T2 Formula (Sheard et al. 2010)
    Modified version of SRK/T formula
    
    Parameters:
    -----------
    AL : float - Axial length (mm)
    K_avg : float - Average keratometry (D)
    IOL_power : float - IOL power (D)
    A_constant : float - A-constant for the IOL
    nc : float - Corneal refractive index (default 1.333)
    k_index : float - Keratometric index (default 1.3375)
    
    Returns:
    --------
    float - Predicted postoperative refraction (D)
    """
    # Constants
    na = 1.336  # Aqueous/vitreous refractive index
    V = 12      # Vertex distance (mm)
    ncm1 = nc - 1
    
    # Calculate corneal radius from keratometry
    r = (k_index - 1) * 1000 / K_avg
    
    # Axial length correction for long eyes
    if AL <= 24.2:
        LCOR = AL
    else:
        LCOR = 3.446 + 1.716 * AL - 0.0237 * AL * AL
    
    # H2 calculation (corneal height) - Sheard's modification
    H2 = -10.326 + 0.32630 * LCOR + 0.13533 * K_avg
    
    # ACD (Anterior Chamber Depth) estimation
    ACD_const = 0.62467 * A_constant - 68.747
    offset = ACD_const - 3.336
    ACD_est = H2 + offset
    
    # Retinal thickness correction
    RETHICK = 0.65696 - 0.02029 * AL
    LOPT = AL + RETHICK  # Optical axial length
    
    # SRK/T2 refraction calculation
    numerator = (1000 * na * (na * r - ncm1 * LOPT) - 
                 IOL_power * (LOPT - ACD_est) * (na * r - ncm1 * ACD_est))
    
    denominator = (na * (V * (na * r - ncm1 * LOPT) + LOPT * r) - 
                   0.001 * IOL_power * (LOPT - ACD_est) * 
                   (V * (na * r - ncm1 * ACD_est) + ACD_est * r))
    
    return numerator / denominator

print("=" * 70)
print("SRK/T2 FORMULA (Sheard et al. 2010)")
print("=" * 70)
print()
print("📐 MAIN FORMULA:")
print("-" * 70)
print()
print("         1000·nₐ·(nₐ·r - nc₋₁·Lopt) - P·(Lopt - ACDest)·(nₐ·r - nc₋₁·ACDest)")
print("REF = ───────────────────────────────────────────────────────────────────────────")
print("       nₐ·(V·(nₐ·r - nc₋₁·Lopt) + Lopt·r) - 0.001·P·(Lopt - ACDest)·(V·(nₐ·r - nc₋₁·ACDest) + ACDest·r)")
print()
print()
print("📖 VARIABLE DEFINITIONS:")
print("=" * 70)
print()
print("INPUT VARIABLES:")
print("-" * 35)
print("• AL         → Axial length of the eye (mm)")
print("• K_avg      → Average keratometry [(Ks + Kf)/2] (diopters)")
print("• IOL_power  → Implanted intraocular lens power (diopters)")
print("• A_constant → IOL-specific A-constant (dimensionless)")
print()
print("PHYSICAL CONSTANTS:")
print("-" * 35)
print("• nₐ = 1.336     → Refractive index of aqueous and vitreous")
print("• nc = 1.333     → Corneal refractive index")
print("• nc₋₁ = 0.333   → nc - 1 (corneal refractive power)")
print("• k_index = 1.3375 → Keratometric index (for K to radius conversion)")
print("• V = 12 mm      → Vertex distance (spectacle-cornea distance)")
print()
print("CALCULATED VARIABLES:")
print("-" * 35)
print("• r          → Corneal radius of curvature (mm)")
print("• LCOR       → Corrected axial length for long eyes (mm)")
print("• H2         → Corneal height according to Sheard (mm)")
print("• ACD_const  → ACD constant derived from A-constant")
print("• offset     → Offset for ACD calculation")
print("• ACDest     → Estimated postoperative anterior chamber depth (mm)")
print("• RETHICK    → Calculated retinal thickness (mm)")
print("• Lopt       → Optical axial length [AL + RETHICK] (mm)")
print("• REF        → Predicted postoperative refraction (diopters)")
print()
print("OTHER SYMBOLS:")
print("-" * 35)
print("• P          → IOL_power (IOL power)")
print("• Ks         → Keratometry flattest meridian (diopters)")
print("• Kf         → Keratometry steepest meridian (diopters)")
print()
print()
print("🔍 INTERMEDIATE CALCULATIONS:")
print("=" * 70)
print()
print("1️⃣  CORNEAL RADIUS (r):")
print("    r = (k_index - 1) × 1000 / K_avg")
print("    where: k_index = 1.3375 (keratometric index)")
print()
print("2️⃣  CORRECTED AXIAL LENGTH (LCOR):")
print("    If AL ≤ 24.2 mm:  LCOR = AL")
print("    If AL > 24.2 mm:  LCOR = 3.446 + 1.716×AL - 0.0237×AL²")
print()
print("3️⃣  CORNEAL HEIGHT H2 (Sheard's modification):")
print("    H2 = -10.326 + 0.32630×LCOR + 0.13533×K_avg")
print()
print("4️⃣  ESTIMATED ANTERIOR CHAMBER DEPTH (ACDest):")
print("    ACD_const = 0.62467×A_constant - 68.747")
print("    offset = ACD_const - 3.336")
print("    ACDest = H2 + offset")
print()
print("5️⃣  OPTICAL AXIAL LENGTH (Lopt):")
print("    RETHICK = 0.65696 - 0.02029×AL  (retinal thickness)")
print("    Lopt = AL + RETHICK")
print()
print()
print("✓ SRK/T2 formula defined and ready for use")

SRK/T2 FORMULA (Sheard et al. 2010)

📐 MAIN FORMULA:
----------------------------------------------------------------------

         1000·nₐ·(nₐ·r - nc₋₁·Lopt) - P·(Lopt - ACDest)·(nₐ·r - nc₋₁·ACDest)
REF = ───────────────────────────────────────────────────────────────────────────
       nₐ·(V·(nₐ·r - nc₋₁·Lopt) + Lopt·r) - 0.001·P·(Lopt - ACDest)·(V·(nₐ·r - nc₋₁·ACDest) + ACDest·r)


📖 VARIABLE DEFINITIONS:

INPUT VARIABLES:
-----------------------------------
• AL         → Axial length of the eye (mm)
• K_avg      → Average keratometry [(Ks + Kf)/2] (diopters)
• IOL_power  → Implanted intraocular lens power (diopters)
• A_constant → IOL-specific A-constant (dimensionless)

PHYSICAL CONSTANTS:
-----------------------------------
• nₐ = 1.336     → Refractive index of aqueous and vitreous
• nc = 1.333     → Corneal refractive index
• nc₋₁ = 0.333   → nc - 1 (corneal refractive power)
• k_index = 1.3375 → Keratometric index (for K to radius conversion)
• V = 12 mm      → Vertex dista

In [3]:
print("CALCULATING SRK/T2 PREDICTIONS...")
print("-" * 70)

# Calculate predictions for all patients
df['SRKT2_Prediction'] = df.apply(
    lambda row: calculate_SRKT2(
        AL=row['Bio-AL'],
        K_avg=row['K_avg'],
        IOL_power=row['IOL Power'],
        A_constant=row['A-Constant']
    ), axis=1
)

# Calculate prediction errors
df['Prediction_Error'] = df['PostOP Spherical Equivalent'] - df['SRKT2_Prediction']
df['Absolute_Error'] = abs(df['Prediction_Error'])

print(f"✓ Predictions calculated for {len(df)} patients")

# Calculate metrics
mae = df['Absolute_Error'].mean()
me = df['Prediction_Error'].mean()
std = df['Prediction_Error'].std()
median_ae = df['Absolute_Error'].median()

print("\n📊 SRK/T2 FORMULA PERFORMANCE METRICS:")
print("=" * 70)
print(f"  Mean Absolute Error (MAE):     {mae:.4f} D")
print(f"  Mean Error (ME):                {me:+.4f} D")
print(f"  Standard Deviation (SD):        {std:.4f} D")
print(f"  Median Absolute Error:          {median_ae:.4f} D")

# Calculate clinical accuracy
within_025 = (df['Absolute_Error'] <= 0.25).sum() / len(df) * 100
within_050 = (df['Absolute_Error'] <= 0.50).sum() / len(df) * 100
within_075 = (df['Absolute_Error'] <= 0.75).sum() / len(df) * 100
within_100 = (df['Absolute_Error'] <= 1.00).sum() / len(df) * 100

print("\n📈 CLINICAL ACCURACY:")
print("-" * 70)
print(f"  Within ±0.25 D:  {within_025:.1f}% of eyes")
print(f"  Within ±0.50 D:  {within_050:.1f}% of eyes")
print(f"  Within ±0.75 D:  {within_075:.1f}% of eyes")
print(f"  Within ±1.00 D:  {within_100:.1f}% of eyes")

CALCULATING SRK/T2 PREDICTIONS...
----------------------------------------------------------------------
✓ Predictions calculated for 96 patients

📊 SRK/T2 FORMULA PERFORMANCE METRICS:
  Mean Absolute Error (MAE):     1.3591 D
  Mean Error (ME):                -0.2915 D
  Standard Deviation (SD):        1.7471 D
  Median Absolute Error:          1.0311 D

📈 CLINICAL ACCURACY:
----------------------------------------------------------------------
  Within ±0.25 D:  13.5% of eyes
  Within ±0.50 D:  26.0% of eyes
  Within ±0.75 D:  35.4% of eyes
  Within ±1.00 D:  49.0% of eyes


In [4]:
# Correlation analysis between MAE and SRK/T2 parameters
print("\n" + "=" * 70)
print("CORRELATION ANALYSIS: MAE vs SRK/T2 PARAMETERS (SPEARMAN)")
print("=" * 70)

# Calculate intermediate parameters used in the formula for each patient
df['r_corneal'] = (1.3375 - 1) * 1000 / df['K_avg']  # Corneal radius

# LCOR (Corrected Axial Length)
df['LCOR'] = df.apply(lambda row: row['Bio-AL'] if row['Bio-AL'] <= 24.2 
                      else 3.446 + 1.716 * row['Bio-AL'] - 0.0237 * row['Bio-AL']**2, 
                      axis=1)

# H2 (Sheard's Corneal Height)
df['H2'] = -10.326 + 0.32630 * df['LCOR'] + 0.13533 * df['K_avg']

# Estimated ACD
df['ACD_const'] = 0.62467 * df['A-Constant'] - 68.747
df['offset'] = df['ACD_const'] - 3.336
df['ACDest'] = df['H2'] + df['offset']

# Retinal thickness and optical length
df['RETHICK'] = 0.65696 - 0.02029 * df['Bio-AL']
df['Lopt'] = df['Bio-AL'] + df['RETHICK']

# Calculate correlations using Spearman method
correlations = {
    'INPUT PARAMETERS': {
        'Axial Length (AL)': df['Bio-AL'].corr(df['Absolute_Error'], method='spearman'),
        'Average Keratometry (K_avg)': df['K_avg'].corr(df['Absolute_Error'], method='spearman'),
        'IOL Power': df['IOL Power'].corr(df['Absolute_Error'], method='spearman'),
        'A-Constant': df['A-Constant'].corr(df['Absolute_Error'], method='spearman'),
        'CCT': df['CCT'].corr(df['Absolute_Error'], method='spearman')
    },
    'CALCULATED PARAMETERS': {
        'Corneal Radius (r)': df['r_corneal'].corr(df['Absolute_Error'], method='spearman'),
        'Corrected AL (LCOR)': df['LCOR'].corr(df['Absolute_Error'], method='spearman'),
        'Corneal Height H2': df['H2'].corr(df['Absolute_Error'], method='spearman'),
        'Estimated ACD': df['ACDest'].corr(df['Absolute_Error'], method='spearman'),
        'Optical Length (Lopt)': df['Lopt'].corr(df['Absolute_Error'], method='spearman'),
        'Retinal Thickness': df['RETHICK'].corr(df['Absolute_Error'], method='spearman')
    }
}

# Print results
print("\n📊 SPEARMAN CORRELATIONS (ρ) WITH ABSOLUTE ERROR:")
print("-" * 70)

for category, params in correlations.items():
    print(f"\n{category}:")
    print("-" * 35)
    for name, corr in sorted(params.items(), key=lambda x: abs(x[1]), reverse=True):
        sign = "+" if corr > 0 else ""
        strength = ""
        abs_corr = abs(corr)
        if abs_corr >= 0.7:
            strength = " [STRONG]"
        elif abs_corr >= 0.5:
            strength = " [MODERATE]"
        elif abs_corr >= 0.3:
            strength = " [WEAK]"
        else:
            strength = " [VERY WEAK]"
        
        print(f"  {name:30} ρ = {sign}{corr:.4f}{strength}")

# Statistical analysis of significant correlations
print("\n📈 INTERPRETATION:")
print("-" * 70)

# Find strongest correlations
all_corrs = []
for cat, params in correlations.items():
    for name, corr in params.items():
        all_corrs.append((name, corr))

all_corrs.sort(key=lambda x: abs(x[1]), reverse=True)
top_3 = all_corrs[:3]

print("\nTOP 3 STRONGEST CORRELATIONS:")
for i, (name, corr) in enumerate(top_3, 1):
    print(f"{i}. {name}: ρ = {corr:+.4f}")
    if corr > 0:
        print(f"   → Higher {name} values associated with larger errors")
    else:
        print(f"   → Higher {name} values associated with smaller errors")

# Significance testing for main correlations
from scipy import stats

print("\n📊 SIGNIFICANCE TESTING (n = 96):")
print("-" * 70)

# Mapping of names to dataframe columns
param_mapping = {
    'Axial Length (AL)': 'Bio-AL',
    'Average Keratometry (K_avg)': 'K_avg',
    'IOL Power': 'IOL Power',
    'A-Constant': 'A-Constant',
    'CCT': 'CCT',
    'Corneal Radius (r)': 'r_corneal',
    'Corrected AL (LCOR)': 'LCOR',
    'Corneal Height H2': 'H2',
    'Estimated ACD': 'ACDest',
    'Optical Length (Lopt)': 'Lopt',
    'Retinal Thickness': 'RETHICK'
}

for name, corr in top_3:
    # Calculate p-value using scipy.stats.spearmanr
    col_name = param_mapping.get(name)
    if col_name:
        rho, p_value = stats.spearmanr(df[col_name], df['Absolute_Error'])
    else:
        p_value = np.nan
    
    sig = ""
    if p_value < 0.001:
        sig = "***"
    elif p_value < 0.01:
        sig = "**"
    elif p_value < 0.05:
        sig = "*"
    else:
        sig = "ns"
    
    print(f"{name:30} p = {p_value:.4f} {sig}")

print("\nLegend: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")


CORRELATION ANALYSIS: MAE vs SRK/T2 PARAMETERS (SPEARMAN)

📊 SPEARMAN CORRELATIONS (ρ) WITH ABSOLUTE ERROR:
----------------------------------------------------------------------

INPUT PARAMETERS:
-----------------------------------
  Axial Length (AL)              ρ = +0.3429 [WEAK]
  IOL Power                      ρ = -0.2460 [VERY WEAK]
  CCT                            ρ = +0.1887 [VERY WEAK]
  Average Keratometry (K_avg)    ρ = -0.1675 [VERY WEAK]
  A-Constant                     ρ = -0.0307 [VERY WEAK]

CALCULATED PARAMETERS:
-----------------------------------
  Corrected AL (LCOR)            ρ = +0.3429 [WEAK]
  Optical Length (Lopt)          ρ = +0.3429 [WEAK]
  Retinal Thickness              ρ = -0.3429 [WEAK]
  Corneal Height H2              ρ = +0.3134 [WEAK]
  Estimated ACD                  ρ = +0.2997 [VERY WEAK]
  Corneal Radius (r)             ρ = +0.1675 [VERY WEAK]

📈 INTERPRETATION:
----------------------------------------------------------------------

TOP 3 STRONG

In [5]:
# Machine Learning Approach to Optimize SRK/T2 Parameters
print("=" * 70)
print("ML-BASED SRK/T2 PARAMETER OPTIMIZATION")
print("=" * 70)

from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Define modified SRK/T2 formula with adjustable parameters
def calculate_SRKT2_modified(AL, K_avg, IOL_power, A_constant, CCT, 
                             nc_adjustment=0, k_index_adjustment=0, acd_offset_adjustment=0):
    """
    Modified SRK/T2 Formula with CCT-based adjustments
    
    Clinically plausible modifications:
    - nc_adjustment: Corneal refractive index change due to DMEK
    - k_index_adjustment: Keratometric index change due to altered posterior/anterior ratio
    - acd_offset_adjustment: ACD estimation adjustment for altered cornea
    """
    # Base constants
    na = 1.336  # Aqueous - NOT MODIFIED (not affected by DMEK)
    V = 12      # Vertex distance - NOT MODIFIED (external to eye)
    
    # Modified parameters based on CCT
    nc = 1.333 + nc_adjustment  # Corneal index affected by hydration
    k_index = 1.3375 + k_index_adjustment  # K-to-radius conversion affected
    
    # Constrain parameters to physical ranges
    nc = np.clip(nc, 1.32, 1.35)
    k_index = np.clip(k_index, 1.32, 1.35)
    
    ncm1 = nc - 1
    
    # Calculate corneal radius with modified k_index
    r = (k_index - 1) * 1000 / K_avg
    
    # Standard LCOR - NOT MODIFIED (axial length is anatomical)
    if AL <= 24.2:
        LCOR = AL
    else:
        LCOR = 3.446 + 1.716 * AL - 0.0237 * AL * AL
    
    # Standard H2 calculation
    H2 = -10.326 + 0.32630 * LCOR + 0.13533 * K_avg
    
    # Modified ACD estimation
    ACD_const = 0.62467 * A_constant - 68.747
    offset = ACD_const - 3.336 + acd_offset_adjustment  # Added adjustment
    ACD_est = H2 + offset
    
    # Standard retinal thickness - NOT MODIFIED (retina not affected by DMEK)
    RETHICK = 0.65696 - 0.02029 * AL
    LOPT = AL + RETHICK
    
    # SRK/T2 calculation with modified parameters
    numerator = (1000 * na * (na * r - ncm1 * LOPT) - 
                 IOL_power * (LOPT - ACD_est) * (na * r - ncm1 * ACD_est))
    
    denominator = (na * (V * (na * r - ncm1 * LOPT) + LOPT * r) - 
                   0.001 * IOL_power * (LOPT - ACD_est) * 
                   (V * (na * r - ncm1 * ACD_est) + ACD_est * r))
    
    return numerator / denominator

print("✓ Modified SRK/T2 formula defined with clinically plausible adjustments")
print()
print("Modifiable parameters (DMEK-affected):")
print("  • nc (corneal refractive index) - range: [1.32, 1.35]")
print("  • k_index (keratometric index) - range: [1.32, 1.35]")
print("  • ACD offset")
print()
print("Fixed parameters (not affected by DMEK):")
print("  • na (aqueous index)")
print("  • RETHICK (retinal thickness)")
print("  • LCOR (axial length correction)")
print("  • V (vertex distance)")

ML-BASED SRK/T2 PARAMETER OPTIMIZATION
✓ Modified SRK/T2 formula defined with clinically plausible adjustments

Modifiable parameters (DMEK-affected):
  • nc (corneal refractive index) - range: [1.32, 1.35]
  • k_index (keratometric index) - range: [1.32, 1.35]
  • ACD offset

Fixed parameters (not affected by DMEK):
  • na (aqueous index)
  • RETHICK (retinal thickness)
  • LCOR (axial length correction)
  • V (vertex distance)


In [6]:
# Feature Engineering for ML Models
print("\n" + "=" * 70)
print("FEATURE ENGINEERING FOR ML OPTIMIZATION")
print("=" * 70)

# Create feature matrix
features = []
feature_names = []

# Primary features
primary_features = ['Bio-AL', 'K_avg', 'IOL Power', 'A-Constant', 'CCT']
features.extend([df[col].values for col in primary_features])
feature_names.extend(primary_features)

# CCT-based features (key for DMEK)
df['CCT_deviation'] = df['CCT'] - 600  # Deviation from "normal" 600μm
df['CCT_squared'] = df['CCT'] ** 2
df['CCT_category'] = pd.cut(df['CCT'], bins=[0, 585, 643, 1000], labels=[0, 1, 2])  # Thin, Normal, Thick

features.extend([
    df['CCT_deviation'].values,
    df['CCT_squared'].values,
    df['CCT_category'].astype(float).values
])
feature_names.extend(['CCT_deviation', 'CCT_squared', 'CCT_category'])

# Interaction terms (clinically meaningful)
df['CCT_x_AL'] = df['CCT'] * df['Bio-AL']
df['CCT_x_K'] = df['CCT'] * df['K_avg']
df['CCT_ratio_AL'] = df['CCT'] / df['Bio-AL']

features.extend([
    df['CCT_x_AL'].values,
    df['CCT_x_K'].values,
    df['CCT_ratio_AL'].values
])
feature_names.extend(['CCT_x_AL', 'CCT_x_K', 'CCT_ratio_AL'])

# Stack features into matrix
X = np.column_stack(features)
y = df['PostOP Spherical Equivalent'].values

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")
print(f"\nFeatures included ({len(feature_names)}):")
for i, name in enumerate(feature_names):
    print(f"  {i+1:2}. {name}")

# Standardize features for ML
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n✓ Features prepared and scaled for ML models")


FEATURE ENGINEERING FOR ML OPTIMIZATION

Feature matrix shape: (96, 11)
Target variable shape: (96,)

Features included (11):
   1. Bio-AL
   2. K_avg
   3. IOL Power
   4. A-Constant
   5. CCT
   6. CCT_deviation
   7. CCT_squared
   8. CCT_category
   9. CCT_x_AL
  10. CCT_x_K
  11. CCT_ratio_AL

✓ Features prepared and scaled for ML models


In [7]:
# K-Fold Cross-Validation ML Models
print("\n" + "=" * 70)
print("K-FOLD CROSS-VALIDATION ML MODELS")
print("=" * 70)

# Set up K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Models to test
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.01, max_iter=10000),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
}

# Store results
results = {}

print("\nTraining models with 10-fold cross-validation...")
print("-" * 70)

for name, model in models.items():
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_scaled, y, cv=kfold, scoring=mae_scorer)
    mae_scores = -cv_scores  # Convert back to positive MAE
    
    results[name] = {
        'mean_mae': mae_scores.mean(),
        'std_mae': mae_scores.std(),
        'all_scores': mae_scores
    }
    
    print(f"\n{name}:")
    print(f"  MAE: {mae_scores.mean():.4f} ± {mae_scores.std():.4f} D")
    print(f"  Min/Max: {mae_scores.min():.4f} / {mae_scores.max():.4f} D")

# Compare with baseline SRK/T2
baseline_mae = df['Absolute_Error'].mean()
print("\n" + "-" * 70)
print(f"Baseline SRK/T2 MAE: {baseline_mae:.4f} D")

# Find best model
best_model_name = min(results, key=lambda x: results[x]['mean_mae'])
best_mae = results[best_model_name]['mean_mae']
improvement = baseline_mae - best_mae
improvement_pct = (improvement / baseline_mae) * 100

print(f"\nBest ML Model: {best_model_name}")
print(f"  MAE: {best_mae:.4f} D")
print(f"  Improvement over SRK/T2: {improvement:.4f} D ({improvement_pct:.1f}%)")


K-FOLD CROSS-VALIDATION ML MODELS

Training models with 10-fold cross-validation...
----------------------------------------------------------------------

Linear Regression:
  MAE: 1.0083 ± 0.2922 D
  Min/Max: 0.5373 / 1.5254 D

Ridge Regression:
  MAE: 0.9431 ± 0.2921 D
  Min/Max: 0.4721 / 1.3983 D

Lasso Regression:
  MAE: 0.9494 ± 0.2882 D
  Min/Max: 0.4696 / 1.3828 D

Random Forest:
  MAE: 0.9733 ± 0.3330 D
  Min/Max: 0.5870 / 1.5184 D

Gradient Boosting:
  MAE: 1.1152 ± 0.2936 D
  Min/Max: 0.6110 / 1.5061 D

----------------------------------------------------------------------
Baseline SRK/T2 MAE: 1.3591 D

Best ML Model: Ridge Regression
  MAE: 0.9431 D
  Improvement over SRK/T2: 0.4160 D (30.6%)


In [8]:
# Ridge-Inspired SRK/T2 Parameter Optimization
print("\n" + "=" * 70)
print("RIDGE-INSPIRED SRK/T2 PARAMETER OPTIMIZATION")
print("=" * 70)

# Step 1: Train Ridge and analyze coefficients
print("\nStep 1: Analyzing Ridge Regression patterns...")
print("-" * 50)

ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_scaled, y)

# Get coefficients and their importance
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': ridge_model.coef_,
    'Abs_Coefficient': np.abs(ridge_model.coef_)
}).sort_values('Abs_Coefficient', ascending=False)

print("\nTop Ridge coefficients (standardized):")
for idx, row in coef_df.head(8).iterrows():
    print(f"{row['Feature']:20} {row['Coefficient']:+.6f}")

# Analyze CCT-related patterns
cct_related = coef_df[coef_df['Feature'].str.contains('CCT')]
print(f"\nCCT-related features total influence: {cct_related['Abs_Coefficient'].sum():.4f}")

# Step 2: Design parameter modifications based on Ridge insights
print("\n" + "=" * 70)
print("Step 2: Translating Ridge patterns into SRK/T2 modifications...")
print("-" * 50)

# Identify key relationships from Ridge
cct_coef = coef_df[coef_df['Feature'] == 'CCT']['Coefficient'].values[0] if 'CCT' in coef_df['Feature'].values else 0
cct_dev_coef = coef_df[coef_df['Feature'] == 'CCT_deviation']['Coefficient'].values[0] if 'CCT_deviation' in coef_df['Feature'].values else 0
cct_k_coef = coef_df[coef_df['Feature'] == 'CCT_x_K']['Coefficient'].values[0] if 'CCT_x_K' in coef_df['Feature'].values else 0
cct_al_coef = coef_df[coef_df['Feature'] == 'CCT_x_AL']['Coefficient'].values[0] if 'CCT_x_AL' in coef_df['Feature'].values else 0

print(f"Key Ridge insights:")
print(f"  CCT main effect:      {cct_coef:+.6f}")
print(f"  CCT deviation effect: {cct_dev_coef:+.6f}")
print(f"  CCT×K interaction:    {cct_k_coef:+.6f}")
print(f"  CCT×AL interaction:   {cct_al_coef:+.6f}")

# Step 3: Enhanced optimization with Ridge-inspired modifications
print("\n" + "=" * 70)
print("Step 3: Optimizing SRK/T2 with Ridge-inspired modifications...")
print("-" * 50)

def calculate_SRKT2_ridge_inspired(AL, K_avg, IOL_power, A_constant, CCT,
                                   alpha_nc, beta_nc_k, gamma_k, delta_acd, epsilon_acd_al):
    """
    SRK/T2 with Ridge-inspired modifications
    
    Based on Ridge patterns:
    - nc affected by CCT and CCT×K interaction
    - k_index affected by CCT
    - ACD offset affected by CCT and CCT×AL interaction
    """
    # Base constants
    na = 1.336
    V = 12
    
    # Ridge-inspired modifications
    cct_norm = (CCT - 600) / 100  # Normalize CCT deviation
    k_norm = (K_avg - 44) / 2     # Normalize K deviation
    al_norm = (AL - 23.5) / 1.5   # Normalize AL deviation
    
    # Modified parameters based on Ridge insights
    nc = 1.333 + alpha_nc * cct_norm + beta_nc_k * cct_norm * k_norm
    k_index = 1.3375 + gamma_k * cct_norm
    
    # Constrain to physical ranges
    nc = np.clip(nc, 1.32, 1.35)
    k_index = np.clip(k_index, 1.32, 1.35)
    
    ncm1 = nc - 1
    
    # Calculate corneal radius
    r = (k_index - 1) * 1000 / K_avg
    
    # Standard LCOR
    if AL <= 24.2:
        LCOR = AL
    else:
        LCOR = 3.446 + 1.716 * AL - 0.0237 * AL * AL
    
    # Standard H2
    H2 = -10.326 + 0.32630 * LCOR + 0.13533 * K_avg
    
    # Modified ACD with Ridge-inspired adjustment
    ACD_const = 0.62467 * A_constant - 68.747
    offset = ACD_const - 3.336 + delta_acd * cct_norm + epsilon_acd_al * cct_norm * al_norm
    ACD_est = H2 + offset
    
    # Standard retinal thickness
    RETHICK = 0.65696 - 0.02029 * AL
    LOPT = AL + RETHICK
    
    # Calculate refraction
    numerator = (1000 * na * (na * r - ncm1 * LOPT) - 
                 IOL_power * (LOPT - ACD_est) * (na * r - ncm1 * ACD_est))
    
    denominator = (na * (V * (na * r - ncm1 * LOPT) + LOPT * r) - 
                   0.001 * IOL_power * (LOPT - ACD_est) * 
                   (V * (na * r - ncm1 * ACD_est) + ACD_est * r))
    
    return numerator / denominator

# Optimize the Ridge-inspired parameters
from scipy.optimize import differential_evolution

def objective_ridge_inspired(params):
    alpha_nc, beta_nc_k, gamma_k, delta_acd, epsilon_acd_al = params
    
    predictions = []
    for idx, row in df.iterrows():
        pred = calculate_SRKT2_ridge_inspired(
            AL=row['Bio-AL'],
            K_avg=row['K_avg'],
            IOL_power=row['IOL Power'],
            A_constant=row['A-Constant'],
            CCT=row['CCT'],
            alpha_nc=alpha_nc,
            beta_nc_k=beta_nc_k,
            gamma_k=gamma_k,
            delta_acd=delta_acd,
            epsilon_acd_al=epsilon_acd_al
        )
        predictions.append(pred)
    
    predictions = np.array(predictions)
    actual = df['PostOP Spherical Equivalent'].values
    mae = np.mean(np.abs(actual - predictions))
    return mae

# Use differential evolution for global optimization
bounds = [
    (-0.01, 0.01),   # alpha_nc: nc base adjustment
    (-0.005, 0.005), # beta_nc_k: nc×K interaction
    (-0.01, 0.01),   # gamma_k: k_index adjustment
    (-0.5, 0.5),     # delta_acd: ACD base adjustment
    (-0.3, 0.3)      # epsilon_acd_al: ACD×AL interaction
]

print("\nOptimizing Ridge-inspired parameters (this may take a minute)...")

result = differential_evolution(
    objective_ridge_inspired,
    bounds,
    seed=42,
    maxiter=30,
    popsize=15,
    disp=False
)

opt_alpha_nc, opt_beta_nc_k, opt_gamma_k, opt_delta_acd, opt_epsilon_acd_al = result.x
optimized_mae_ridge = result.fun

print("\n" + "=" * 70)
print("OPTIMIZATION RESULTS:")
print("-" * 70)
print("\nOptimal Ridge-inspired parameters:")
print(f"  α_nc (nc base adjustment):        {opt_alpha_nc:.6f}")
print(f"  β_nc_k (nc×K interaction):        {opt_beta_nc_k:.6f}")
print(f"  γ_k (k_index adjustment):         {opt_gamma_k:.6f}")
print(f"  δ_acd (ACD base adjustment):      {opt_delta_acd:.4f} mm")
print(f"  ε_acd_al (ACD×AL interaction):    {opt_epsilon_acd_al:.4f} mm")

print(f"\nPerformance:")
print(f"  Ridge-inspired SRK/T2 MAE:  {optimized_mae_ridge:.4f} D")
print(f"  Original SRK/T2 MAE:         {baseline_mae:.4f} D")
print(f"  Pure Ridge MAE (CV):         {results['Ridge Regression']['mean_mae']:.4f} D")

improvement_ridge = baseline_mae - optimized_mae_ridge
improvement_pct_ridge = (improvement_ridge / baseline_mae) * 100

print(f"\nImprovement over original SRK/T2: {improvement_ridge:.4f} D ({improvement_pct_ridge:.1f}%)")
print(f"Captures {improvement_pct_ridge/30.6*100:.0f}% of Ridge's improvement while keeping SRK/T2 structure")

# Create interpretable formulas
print("\n" + "=" * 70)
print("INTERPRETABLE RIDGE-INSPIRED SRK/T2 FORMULAS:")
print("-" * 70)
print("\nFor DMEK patients, modify SRK/T2 parameters as follows:")
print()
print("Let:")
print("  CCT_norm = (CCT - 600) / 100")
print("  K_norm = (K_avg - 44) / 2")
print("  AL_norm = (AL - 23.5) / 1.5")
print()
print("Then:")
print(f"  nc = 1.333 + {opt_alpha_nc:.6f}×CCT_norm + {opt_beta_nc_k:.6f}×CCT_norm×K_norm")
print(f"  k_index = 1.3375 + {opt_gamma_k:.6f}×CCT_norm")
print(f"  ACD_offset = standard + {opt_delta_acd:.4f}×CCT_norm + {opt_epsilon_acd_al:.4f}×CCT_norm×AL_norm")


RIDGE-INSPIRED SRK/T2 PARAMETER OPTIMIZATION

Step 1: Analyzing Ridge Regression patterns...
--------------------------------------------------

Top Ridge coefficients (standardized):
CCT_ratio_AL         +1.327950
CCT_x_AL             -0.936582
CCT_squared          -0.583618
Bio-AL               +0.528939
K_avg                -0.380900
CCT_x_K              +0.251854
CCT_category         +0.081634
IOL Power            -0.047962

CCT-related features total influence: 3.2170

Step 2: Translating Ridge patterns into SRK/T2 modifications...
--------------------------------------------------
Key Ridge insights:
  CCT main effect:      -0.017657
  CCT deviation effect: -0.017657
  CCT×K interaction:    +0.251854
  CCT×AL interaction:   -0.936582

Step 3: Optimizing SRK/T2 with Ridge-inspired modifications...
--------------------------------------------------

Optimizing Ridge-inspired parameters (this may take a minute)...

OPTIMIZATION RESULTS:
---------------------------------------------

In [9]:
# Enhanced SRK/T2 with Additive Correction Term
print("\n" + "=" * 70)
print("ENHANCED SRK/T2 WITH ADDITIVE CORRECTION TERM")
print("=" * 70)

# Based on Ridge analysis, the most important features are:
# CCT_ratio_AL, CCT_x_AL, CCT_squared, Bio-AL

print("\nApproach: Add a correction term to standard SRK/T2")
print("REF_final = REF_SRKT2 + Correction_term")
print("\nwhere Correction_term captures CCT-related patterns from Ridge")

# Define enhanced formula with correction term
def calculate_SRKT2_enhanced(AL, K_avg, IOL_power, A_constant, CCT,
                             a0, a1, a2, a3, a4, a5):
    """
    Enhanced SRK/T2 with additive correction term
    REF = SRKT2_standard + correction_term
    
    Correction term based on Ridge's top features:
    - CCT/AL ratio
    - CCT×AL interaction  
    - CCT squared
    - CCT deviation
    """
    # First calculate standard SRK/T2
    ref_standard = calculate_SRKT2(AL, K_avg, IOL_power, A_constant)
    
    # Normalize features (same as Ridge)
    cct_norm = (CCT - 600) / 100
    al_norm = (AL - 23.5) / 1.5
    k_norm = (K_avg - 44) / 2
    
    # Correction term inspired by Ridge's top features
    correction = (a0 +                           # Intercept
                 a1 * (CCT/AL - 26) +            # CCT/AL ratio (normalized)
                 a2 * cct_norm * al_norm +       # CCT×AL interaction
                 a3 * cct_norm**2 +              # CCT squared
                 a4 * cct_norm +                 # CCT main effect
                 a5 * cct_norm * k_norm)         # CCT×K interaction
    
    return ref_standard + correction

# Optimize correction term parameters
print("\nOptimizing correction term parameters...")
print("-" * 50)

from scipy.optimize import differential_evolution

def objective_enhanced(params):
    predictions = []
    for idx, row in df.iterrows():
        pred = calculate_SRKT2_enhanced(
            AL=row['Bio-AL'],
            K_avg=row['K_avg'],
            IOL_power=row['IOL Power'],
            A_constant=row['A-Constant'],
            CCT=row['CCT'],
            a0=params[0], a1=params[1], a2=params[2],
            a3=params[3], a4=params[4], a5=params[5]
        )
        predictions.append(pred)
    
    predictions = np.array(predictions)
    actual = df['PostOP Spherical Equivalent'].values
    mae = np.mean(np.abs(actual - predictions))
    return mae

# Bounds for correction parameters
bounds = [
    (-1.0, 1.0),   # a0: intercept
    (-2.0, 2.0),   # a1: CCT/AL ratio coefficient
    (-2.0, 2.0),   # a2: CCT×AL interaction
    (-2.0, 2.0),   # a3: CCT squared
    (-2.0, 2.0),   # a4: CCT main effect
    (-2.0, 2.0),   # a5: CCT×K interaction
]

result_enhanced = differential_evolution(
    objective_enhanced,
    bounds,
    seed=42,
    maxiter=50,
    popsize=20,
    disp=False
)

# Extract optimal parameters
opt_params = result_enhanced.x
enhanced_mae = result_enhanced.fun

print("\nOptimal correction term parameters:")
print(f"  a0 (intercept):       {opt_params[0]:+.4f}")
print(f"  a1 (CCT/AL ratio):    {opt_params[1]:+.4f}")
print(f"  a2 (CCT×AL):          {opt_params[2]:+.4f}")
print(f"  a3 (CCT²):            {opt_params[3]:+.4f}")
print(f"  a4 (CCT):             {opt_params[4]:+.4f}")
print(f"  a5 (CCT×K):           {opt_params[5]:+.4f}")

print("\n" + "=" * 70)
print("PERFORMANCE COMPARISON:")
print("-" * 70)
print(f"  Original SRK/T2 MAE:             {baseline_mae:.4f} D")
print(f"  Ridge-inspired SRK/T2 MAE:       {optimized_mae_ridge:.4f} D")
print(f"  Enhanced SRK/T2 (additive) MAE:  {enhanced_mae:.4f} D")
print(f"  Pure Ridge MAE (CV):             {results['Ridge Regression']['mean_mae']:.4f} D")

improvement_enhanced = baseline_mae - enhanced_mae
improvement_pct_enhanced = (improvement_enhanced / baseline_mae) * 100

print(f"\nEnhanced formula improvement: {improvement_enhanced:.4f} D ({improvement_pct_enhanced:.1f}%)")
print(f"Captures {improvement_pct_enhanced/30.6*100:.0f}% of Ridge's improvement")

# Create final formula
print("\n" + "=" * 70)
print("FINAL ENHANCED SRK/T2-DMEK FORMULA:")
print("-" * 70)
print("\nREF = SRKT2_standard + Correction")
print("\nwhere Correction =")
print(f"  {opt_params[0]:+.4f}")
print(f"  {opt_params[1]:+.4f} × (CCT/AL - 26)")
print(f"  {opt_params[2]:+.4f} × [(CCT-600)/100] × [(AL-23.5)/1.5]")
print(f"  {opt_params[3]:+.4f} × [(CCT-600)/100]²")
print(f"  {opt_params[4]:+.4f} × [(CCT-600)/100]")
print(f"  {opt_params[5]:+.4f} × [(CCT-600)/100] × [(K-44)/2]")

# Alternative: Try multiplicative correction
print("\n" + "=" * 70)
print("ALTERNATIVE: MULTIPLICATIVE CORRECTION")
print("-" * 70)

def calculate_SRKT2_multiplicative(AL, K_avg, IOL_power, A_constant, CCT, m0, m1, m2):
    """
    SRK/T2 with multiplicative correction
    REF = SRKT2_standard × (1 + correction_factor)
    """
    ref_standard = calculate_SRKT2(AL, K_avg, IOL_power, A_constant)
    
    cct_norm = (CCT - 600) / 100
    al_norm = (AL - 23.5) / 1.5
    
    # Multiplicative factor
    factor = 1 + m0 + m1 * cct_norm + m2 * (CCT/AL - 26)
    
    return ref_standard * factor

def objective_multiplicative(params):
    predictions = []
    for idx, row in df.iterrows():
        pred = calculate_SRKT2_multiplicative(
            AL=row['Bio-AL'],
            K_avg=row['K_avg'],
            IOL_power=row['IOL Power'],
            A_constant=row['A-Constant'],
            CCT=row['CCT'],
            m0=params[0], m1=params[1], m2=params[2]
        )
        predictions.append(pred)
    
    predictions = np.array(predictions)
    actual = df['PostOP Spherical Equivalent'].values
    mae = np.mean(np.abs(actual - predictions))
    return mae

bounds_mult = [(-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5)]

result_mult = differential_evolution(
    objective_multiplicative,
    bounds_mult,
    seed=42,
    maxiter=30,
    disp=False
)

mult_params = result_mult.x
mult_mae = result_mult.fun

print(f"\nMultiplicative correction MAE: {mult_mae:.4f} D")
print(f"Improvement: {baseline_mae - mult_mae:.4f} D ({(baseline_mae - mult_mae)/baseline_mae*100:.1f}%)")

print("\nMultiplicative formula:")
print(f"REF = SRKT2 × (1 + {mult_params[0]:+.4f} + {mult_params[1]:+.4f}×CCT_norm + {mult_params[2]:+.4f}×(CCT/AL-26))")


ENHANCED SRK/T2 WITH ADDITIVE CORRECTION TERM

Approach: Add a correction term to standard SRK/T2
REF_final = REF_SRKT2 + Correction_term

where Correction_term captures CCT-related patterns from Ridge

Optimizing correction term parameters...
--------------------------------------------------

Optimal correction term parameters:
  a0 (intercept):       +0.0730
  a1 (CCT/AL ratio):    +0.4100
  a2 (CCT×AL):          +0.0829
  a3 (CCT²):            -0.2705
  a4 (CCT):             -1.6048
  a5 (CCT×K):           +0.2168

PERFORMANCE COMPARISON:
----------------------------------------------------------------------
  Original SRK/T2 MAE:             1.3591 D
  Ridge-inspired SRK/T2 MAE:       1.3184 D
  Enhanced SRK/T2 (additive) MAE:  1.1967 D
  Pure Ridge MAE (CV):             0.9431 D

Enhanced formula improvement: 0.1624 D (12.0%)
Captures 39% of Ridge's improvement

FINAL ENHANCED SRK/T2-DMEK FORMULA:
----------------------------------------------------------------------

REF = SRKT

In [10]:
# Extended Range Optimization for Post-DMEK Corneal Parameters
print("\n" + "=" * 70)
print("EXTENDED RANGE OPTIMIZATION FOR POST-DMEK PARAMETERS")
print("=" * 70)

print("\nRationale for wider ranges:")
print("-" * 50)
print("• Post-DMEK corneas have altered hydration states")
print("• Graft-host interface creates optical discontinuity")
print("• Posterior/anterior curvature ratio significantly changed")
print("• Standard keratometric assumptions may not hold")

# Define SRK/T2 with wider parameter ranges
def calculate_SRKT2_extended_range(AL, K_avg, IOL_power, A_constant, CCT,
                                   nc_base, nc_cct_coef, k_index_base, k_index_cct_coef,
                                   acd_offset_base, acd_offset_cct_coef):
    """
    SRK/T2 with extended parameter ranges for post-DMEK
    
    Wider ranges:
    - nc: 1.25 to 1.40 (vs standard 1.333)
    - k_index: 1.25 to 1.45 (vs standard 1.3375)
    - Larger ACD offset adjustments
    """
    na = 1.336
    V = 12
    
    # CCT-dependent parameters with wider ranges
    cct_norm = (CCT - 600) / 100
    
    nc = nc_base + nc_cct_coef * cct_norm
    k_index = k_index_base + k_index_cct_coef * cct_norm
    
    # Extended ranges for post-DMEK
    nc = np.clip(nc, 1.25, 1.40)  # Much wider range
    k_index = np.clip(k_index, 1.25, 1.45)  # Much wider range
    
    ncm1 = nc - 1
    
    # Calculate with modified parameters
    r = (k_index - 1) * 1000 / K_avg
    
    if AL <= 24.2:
        LCOR = AL
    else:
        LCOR = 3.446 + 1.716 * AL - 0.0237 * AL * AL
    
    H2 = -10.326 + 0.32630 * LCOR + 0.13533 * K_avg
    
    ACD_const = 0.62467 * A_constant - 68.747
    offset = ACD_const - 3.336 + acd_offset_base + acd_offset_cct_coef * cct_norm
    ACD_est = H2 + offset
    
    RETHICK = 0.65696 - 0.02029 * AL
    LOPT = AL + RETHICK
    
    numerator = (1000 * na * (na * r - ncm1 * LOPT) - 
                 IOL_power * (LOPT - ACD_est) * (na * r - ncm1 * ACD_est))
    
    denominator = (na * (V * (na * r - ncm1 * LOPT) + LOPT * r) - 
                   0.001 * IOL_power * (LOPT - ACD_est) * 
                   (V * (na * r - ncm1 * ACD_est) + ACD_est * r))
    
    return numerator / denominator

# Optimize with extended ranges
print("\nOptimizing with extended parameter ranges...")
print("-" * 50)

def objective_extended(params):
    predictions = []
    for idx, row in df.iterrows():
        pred = calculate_SRKT2_extended_range(
            AL=row['Bio-AL'],
            K_avg=row['K_avg'],
            IOL_power=row['IOL Power'],
            A_constant=row['A-Constant'],
            CCT=row['CCT'],
            nc_base=params[0],
            nc_cct_coef=params[1],
            k_index_base=params[2],
            k_index_cct_coef=params[3],
            acd_offset_base=params[4],
            acd_offset_cct_coef=params[5]
        )
        predictions.append(pred)
    
    predictions = np.array(predictions)
    actual = df['PostOP Spherical Equivalent'].values
    mae = np.mean(np.abs(actual - predictions))
    return mae

# Extended bounds
bounds_extended = [
    (1.25, 1.40),    # nc_base (wider: 1.25-1.40 vs 1.32-1.35)
    (-0.05, 0.05),   # nc_cct_coef (5x larger range)
    (1.25, 1.45),    # k_index_base (wider: 1.25-1.45 vs 1.32-1.35)
    (-0.10, 0.10),   # k_index_cct_coef (10x larger range)
    (-1.0, 1.0),     # acd_offset_base (2x larger)
    (-1.0, 1.0),     # acd_offset_cct_coef (2x larger)
]

from scipy.optimize import differential_evolution
import time

# Add callback for progress monitoring
iteration_count = [0]
start_time = time.time()

def callback_extended(xk, convergence):
    iteration_count[0] += 1
    if iteration_count[0] % 10 == 0:
        elapsed = time.time() - start_time
        print(f"  Iteration {iteration_count[0]}: convergence = {convergence:.6f}, time = {elapsed:.1f}s")
    return False

print("Starting FULL optimization (may take 2-5 minutes)...")
print("Progress updates every 10 iterations:")

result_extended = differential_evolution(
    objective_extended,
    bounds_extended,
    seed=42,
    maxiter=100,     # Full iterations as originally intended
    popsize=30,      # Full population as originally intended 
    disp=False,
    workers=1,       # Single thread for callback to work
    callback=callback_extended
)

extended_params = result_extended.x
extended_mae = result_extended.fun

print(f"\nOptimization completed in {time.time() - start_time:.1f} seconds")
print(f"Total iterations: {iteration_count[0]}")
print(f"Total function evaluations: ~{iteration_count[0] * 30} (iterations × population)")

print("\nOptimal parameters with extended ranges:")
print(f"  nc_base:           {extended_params[0]:.4f} (standard: 1.333)")
print(f"  nc_cct_coef:       {extended_params[1]:+.4f}")
print(f"  k_index_base:      {extended_params[2]:.4f} (standard: 1.3375)")
print(f"  k_index_cct_coef:  {extended_params[3]:+.4f}")
print(f"  acd_offset_base:   {extended_params[4]:+.4f} mm")
print(f"  acd_offset_cct_coef: {extended_params[5]:+.4f} mm")

# Calculate effective ranges for typical CCT values
cct_thin = 550  # Thin post-DMEK
cct_thick = 650  # Thick post-DMEK

nc_thin = extended_params[0] + extended_params[1] * (cct_thin - 600) / 100
nc_thick = extended_params[0] + extended_params[1] * (cct_thick - 600) / 100
k_thin = extended_params[2] + extended_params[3] * (cct_thin - 600) / 100
k_thick = extended_params[2] + extended_params[3] * (cct_thick - 600) / 100

print("\nEffective parameter ranges for CCT 550-650 μm:")
print(f"  nc range:      {min(nc_thin, nc_thick):.4f} to {max(nc_thin, nc_thick):.4f}")
print(f"  k_index range: {min(k_thin, k_thick):.4f} to {max(k_thin, k_thick):.4f}")

print("\n" + "=" * 70)
print("PERFORMANCE COMPARISON:")
print("-" * 70)
print(f"  Original SRK/T2 MAE:              {baseline_mae:.4f} D")
print(f"  Conservative range MAE:           {optimized_mae_ridge:.4f} D ({(baseline_mae-optimized_mae_ridge)/baseline_mae*100:.1f}% improvement)")
print(f"  Extended range MAE:               {extended_mae:.4f} D ({(baseline_mae-extended_mae)/baseline_mae*100:.1f}% improvement)")
print(f"  Additive correction MAE:          {enhanced_mae:.4f} D ({(baseline_mae-enhanced_mae)/baseline_mae*100:.1f}% improvement)")
print(f"  Multiplicative correction MAE:    {mult_mae:.4f} D ({(baseline_mae-mult_mae)/baseline_mae*100:.1f}% improvement)")
print(f"  Pure Ridge MAE (theoretical):     {results['Ridge Regression']['mean_mae']:.4f} D ({(baseline_mae-results['Ridge Regression']['mean_mae'])/baseline_mae*100:.1f}% improvement)")

# Test if parameters are at boundaries
print("\n" + "=" * 70)
print("PARAMETER BOUNDARY ANALYSIS:")
print("-" * 70)

for i, (param, bound, name) in enumerate(zip(extended_params, bounds_extended, 
                                             ['nc_base', 'nc_cct_coef', 'k_index_base', 
                                              'k_index_cct_coef', 'acd_offset_base', 'acd_offset_cct_coef'])):
    at_lower = abs(param - bound[0]) < 0.001
    at_upper = abs(param - bound[1]) < 0.001
    if at_lower or at_upper:
        print(f"  {name}: {param:.4f} - AT {'LOWER' if at_lower else 'UPPER'} BOUNDARY [{bound[0]:.2f}, {bound[1]:.2f}]")
    else:
        print(f"  {name}: {param:.4f} - within bounds [{bound[0]:.2f}, {bound[1]:.2f}]")

if any(abs(p - b[0]) < 0.001 or abs(p - b[1]) < 0.001 for p, b in zip(extended_params, bounds_extended)):
    print("\n⚠️ Some parameters at boundaries - consider expanding ranges further")
else:
    print("\n✓ All parameters within bounds - optimization likely found true optimum")


EXTENDED RANGE OPTIMIZATION FOR POST-DMEK PARAMETERS

Rationale for wider ranges:
--------------------------------------------------
• Post-DMEK corneas have altered hydration states
• Graft-host interface creates optical discontinuity
• Posterior/anterior curvature ratio significantly changed
• Standard keratometric assumptions may not hold

Optimizing with extended parameter ranges...
--------------------------------------------------
Starting FULL optimization (may take 2-5 minutes)...
Progress updates every 10 iterations:
  Iteration 10: convergence = 0.045369, time = 6.0s
  Iteration 20: convergence = 0.101280, time = 11.5s
  Iteration 30: convergence = 0.221439, time = 17.0s
  Iteration 40: convergence = 0.508274, time = 22.5s
  Iteration 50: convergence = 1.007479, time = 27.9s

Optimization completed in 29.0 seconds
Total iterations: 50
Total function evaluations: ~1500 (iterations × population)

Optimal parameters with extended ranges:
  nc_base:           1.3598 (standard: 1

In [11]:
# Ultra-wide Range Exploration
print("\n" + "=" * 70)
print("ULTRA-WIDE RANGE EXPLORATION FOR PRE-DMEK CORNEAL STATES")
print("=" * 70)

print("\nRationale: Pre-DMEK corneas have extreme optical alterations due to:")
print("-" * 50)
print("• Severe corneal edema from endothelial dysfunction")
print("• Fuchs' dystrophy causing irregular hydration")
print("• Descemet's membrane irregularities")
print("• Significant posterior surface changes")
print("\nThese alterations will be corrected by DMEK, but IOL calculation")
print("must account for the current (pre-surgical) abnormal state.")

# Ultra-wide bounds - exploring full physical possibilities
bounds_ultra = [
    (1.20, 1.50),    # nc_base (ultra-wide: edematous cornea can vary greatly)
    (-0.20, 0.20),   # nc_cct_coef (very large CCT influence due to edema)
    (1.20, 1.60),    # k_index_base (exploring full range for diseased corneas)
    (-0.30, 0.30),   # k_index_cct_coef (massive CCT effect in edematous corneas)
    (-3.0, 3.0),     # acd_offset_base (extreme ACD changes)
    (-3.0, 3.0),     # acd_offset_cct_coef (extreme CCT-ACD coupling)
]

print("\nTesting bounds for pre-DMEK diseased corneas:")
for i, (bound, name) in enumerate(zip(bounds_ultra, 
                                       ['nc_base', 'nc_cct_coef', 'k_index_base', 
                                        'k_index_cct_coef', 'acd_offset_base', 'acd_offset_cct_coef'])):
    print(f"  {name:20} [{bound[0]:+.2f}, {bound[1]:+.2f}]")

# Modify function to allow ultra-wide ranges for pre-DMEK corneas
def calculate_SRKT2_ultra_range(AL, K_avg, IOL_power, A_constant, CCT,
                                nc_base, nc_cct_coef, k_index_base, k_index_cct_coef,
                                acd_offset_base, acd_offset_cct_coef):
    """
    SRK/T2 for pre-DMEK corneas with extreme optical alterations
    
    The cornea measured pre-operatively has:
    - Severe edema (high CCT)
    - Altered refractive indices
    - Irregular posterior surface
    
    These will normalize after DMEK, but current measurements
    reflect the diseased state.
    """
    na = 1.336
    V = 12
    
    cct_norm = (CCT - 600) / 100
    
    nc = nc_base + nc_cct_coef * cct_norm
    k_index = k_index_base + k_index_cct_coef * cct_norm
    
    # Ultra-wide ranges for diseased corneas
    nc = np.clip(nc, 1.15, 1.55)
    k_index = np.clip(k_index, 1.15, 1.65)
    
    ncm1 = nc - 1
    
    r = (k_index - 1) * 1000 / K_avg
    
    if AL <= 24.2:
        LCOR = AL
    else:
        LCOR = 3.446 + 1.716 * AL - 0.0237 * AL * AL
    
    H2 = -10.326 + 0.32630 * LCOR + 0.13533 * K_avg
    
    ACD_const = 0.62467 * A_constant - 68.747
    offset = ACD_const - 3.336 + acd_offset_base + acd_offset_cct_coef * cct_norm
    ACD_est = H2 + offset
    
    RETHICK = 0.65696 - 0.02029 * AL
    LOPT = AL + RETHICK
    
    numerator = (1000 * na * (na * r - ncm1 * LOPT) - 
                 IOL_power * (LOPT - ACD_est) * (na * r - ncm1 * ACD_est))
    
    denominator = (na * (V * (na * r - ncm1 * LOPT) + LOPT * r) - 
                   0.001 * IOL_power * (LOPT - ACD_est) * 
                   (V * (na * r - ncm1 * ACD_est) + ACD_est * r))
    
    return numerator / denominator

def objective_ultra(params):
    predictions = []
    for idx, row in df.iterrows():
        pred = calculate_SRKT2_ultra_range(
            AL=row['Bio-AL'],
            K_avg=row['K_avg'],
            IOL_power=row['IOL Power'],
            A_constant=row['A-Constant'],
            CCT=row['CCT'],
            nc_base=params[0],
            nc_cct_coef=params[1],
            k_index_base=params[2],
            k_index_cct_coef=params[3],
            acd_offset_base=params[4],
            acd_offset_cct_coef=params[5]
        )
        predictions.append(pred)
    
    predictions = np.array(predictions)
    actual = df['PostOP Spherical Equivalent'].values
    mae = np.mean(np.abs(actual - predictions))
    return mae

print("\nOptimizing for pre-DMEK edematous corneas...")
print("-" * 50)

import time
from scipy.optimize import differential_evolution

# Add callback for progress monitoring
iteration_count_ultra = [0]
start_time_ultra = time.time()

def callback_ultra(xk, convergence):
    iteration_count_ultra[0] += 1
    if iteration_count_ultra[0] % 10 == 0:
        elapsed = time.time() - start_time_ultra
        print(f"  Iteration {iteration_count_ultra[0]}: convergence = {convergence:.6f}, time = {elapsed:.1f}s")
    return False

print("Starting ULTRA-WIDE optimization (may take 3-6 minutes)...")
print("Progress updates every 10 iterations:")

result_ultra = differential_evolution(
    objective_ultra,
    bounds_ultra,
    seed=42,
    maxiter=150,
    popsize=40,
    disp=False,
    workers=1,      # Single thread for callback to work
    callback=callback_ultra
)

ultra_params = result_ultra.x
ultra_mae = result_ultra.fun

print(f"\nOptimization completed in {time.time() - start_time_ultra:.1f} seconds")
print(f"Total iterations: {iteration_count_ultra[0]}")
print(f"Total function evaluations: ~{iteration_count_ultra[0] * 40} (iterations × population)")

print("\n" + "=" * 70)
print("ULTRA-WIDE OPTIMIZATION RESULTS:")
print("-" * 70)

print("\nOptimal parameters for pre-DMEK corneas:")
print(f"  nc_base:             {ultra_params[0]:.4f} (standard: 1.333)")
print(f"  nc_cct_coef:         {ultra_params[1]:+.4f}")
print(f"  k_index_base:        {ultra_params[2]:.4f} (standard: 1.3375)")
print(f"  k_index_cct_coef:    {ultra_params[3]:+.4f}")
print(f"  acd_offset_base:     {ultra_params[4]:+.4f} mm")
print(f"  acd_offset_cct_coef: {ultra_params[5]:+.4f} mm")

# Check boundaries
print("\nBoundary analysis:")
for i, (param, bound, name) in enumerate(zip(ultra_params, bounds_ultra, 
                                             ['nc_base', 'nc_cct_coef', 'k_index_base', 
                                              'k_index_cct_coef', 'acd_offset_base', 'acd_offset_cct_coef'])):
    at_lower = abs(param - bound[0]) < 0.001
    at_upper = abs(param - bound[1]) < 0.001
    if at_lower or at_upper:
        print(f"  ⚠️ {name}: {param:.4f} - AT {'LOWER' if at_lower else 'UPPER'} BOUNDARY")
    else:
        print(f"  ✓ {name}: {param:.4f} - within bounds")

# Calculate physical interpretation for edematous corneas
cct_mild_edema = 580   # Mild edema
cct_severe_edema = 650  # Severe edema

nc_mild = ultra_params[0] + ultra_params[1] * (cct_mild_edema - 600) / 100
nc_severe = ultra_params[0] + ultra_params[1] * (cct_severe_edema - 600) / 100
k_mild = ultra_params[2] + ultra_params[3] * (cct_mild_edema - 600) / 100
k_severe = ultra_params[2] + ultra_params[3] * (cct_severe_edema - 600) / 100

print("\nPhysical parameters for edematous pre-DMEK corneas:")
print(f"  Mild edema (CCT=580μm):")
print(f"    nc: {nc_mild:.4f}, k_index: {k_mild:.4f}")
print(f"  Severe edema (CCT=650μm):")
print(f"    nc: {nc_severe:.4f}, k_index: {k_severe:.4f}")

# Final comparison
print("\n" + "=" * 70)
print("FINAL PERFORMANCE RANKING:")
print("-" * 70)

methods = [
    ("Original SRK/T2", baseline_mae, 0),
    ("Conservative optimization", optimized_mae_ridge, (baseline_mae-optimized_mae_ridge)/baseline_mae*100),
    ("Extended range optimization", extended_mae, (baseline_mae-extended_mae)/baseline_mae*100),
    ("Ultra-wide range (pre-DMEK)", ultra_mae, (baseline_mae-ultra_mae)/baseline_mae*100),
    ("Additive correction", enhanced_mae, (baseline_mae-enhanced_mae)/baseline_mae*100),
    ("Multiplicative correction", mult_mae, (baseline_mae-mult_mae)/baseline_mae*100),
    ("Pure ML (Ridge)", results['Ridge Regression']['mean_mae'], 
     (baseline_mae-results['Ridge Regression']['mean_mae'])/baseline_mae*100)
]

methods.sort(key=lambda x: x[1])

print(f"{'Rank':<5} {'Method':<35} {'MAE (D)':<10} {'Improvement':<12}")
print("-" * 65)
for i, (name, mae, improvement) in enumerate(methods, 1):
    print(f"{i:<5} {name:<35} {mae:<10.4f} {improvement:>10.1f}%")

best_method = methods[0]
print(f"\n🏆 Best method: {best_method[0]}")
print(f"   MAE: {best_method[1]:.4f} D ({best_method[2]:.1f}% improvement)")

# Calculate percentage within clinical targets
if "Ultra-wide" in best_method[0]:
    best_predictions = []
    for idx, row in df.iterrows():
        pred = calculate_SRKT2_ultra_range(
            AL=row['Bio-AL'], K_avg=row['K_avg'], 
            IOL_power=row['IOL Power'], A_constant=row['A-Constant'],
            CCT=row['CCT'], nc_base=ultra_params[0], nc_cct_coef=ultra_params[1],
            k_index_base=ultra_params[2], k_index_cct_coef=ultra_params[3],
            acd_offset_base=ultra_params[4], acd_offset_cct_coef=ultra_params[5]
        )
        best_predictions.append(pred)
    
    best_errors = np.abs(df['PostOP Spherical Equivalent'].values - np.array(best_predictions))
    within_05 = (best_errors <= 0.5).mean() * 100
    within_10 = (best_errors <= 1.0).mean() * 100
    
    print(f"\n   Clinical accuracy:")
    print(f"   Within ±0.5 D: {within_05:.1f}% (vs {(df['Absolute_Error'] <= 0.5).mean()*100:.1f}% original)")
    print(f"   Within ±1.0 D: {within_10:.1f}% (vs {(df['Absolute_Error'] <= 1.0).mean()*100:.1f}% original)")

print("\n" + "=" * 70)
print("CLINICAL INTERPRETATION:")
print("-" * 70)
print("The optimized parameters suggest that pre-DMEK corneas with")
print("endothelial dysfunction have significantly altered optical properties")
print("that standard IOL formulas fail to account for. The edematous state")
print("changes both the refractive index and the keratometric relationship,")
print("requiring substantial adjustments to achieve accurate IOL power calculation.")


ULTRA-WIDE RANGE EXPLORATION FOR PRE-DMEK CORNEAL STATES

Rationale: Pre-DMEK corneas have extreme optical alterations due to:
--------------------------------------------------
• Severe corneal edema from endothelial dysfunction
• Fuchs' dystrophy causing irregular hydration
• Descemet's membrane irregularities
• Significant posterior surface changes

These alterations will be corrected by DMEK, but IOL calculation
must account for the current (pre-surgical) abnormal state.

Testing bounds for pre-DMEK diseased corneas:
  nc_base              [+1.20, +1.50]
  nc_cct_coef          [-0.20, +0.20]
  k_index_base         [+1.20, +1.60]
  k_index_cct_coef     [-0.30, +0.30]
  acd_offset_base      [-3.00, +3.00]
  acd_offset_cct_coef  [-3.00, +3.00]

Optimizing for pre-DMEK edematous corneas...
--------------------------------------------------
Starting ULTRA-WIDE optimization (may take 3-6 minutes)...
Progress updates every 10 iterations:
  Iteration 10: convergence = 0.019828, time = 8.7

In [ ]:
# COMPREHENSIVE SUMMARY OF ALL METHODS
print("=" * 80)
print("COMPLETE SUMMARY: IOL FORMULA OPTIMIZATION FOR PRE-DMEK PATIENTS")
print("=" * 80)

print("\n📊 DATASET:")
print("-" * 40)
print(f"• 96 patients undergoing combined phaco-DMEK surgery")
print(f"• Pre-operative measurements (edematous corneas with Fuchs' dystrophy)")
print(f"• Post-operative refractive outcomes")
print(f"• Key finding: High CCT (mean ~{df['CCT'].mean():.0f}μm) due to corneal edema")

print("\n" + "=" * 80)
print("METHODS ATTEMPTED (IN CHRONOLOGICAL ORDER):")
print("=" * 80)

methods_summary = """
1. BASELINE: STANDARD SRK/T2
   - Original Sheard et al. (2010) formula
   - Uses: AL, K_avg, IOL_power, A_constant
   - IGNORES: CCT (critical for DMEK patients)
   - MAE: 1.3591 D
   - Only 49% within ±1.0 D

2. MACHINE LEARNING EXPLORATION
   Purpose: Understand theoretical best possible performance
   
   Models tested with k-fold cross-validation:
   • Linear Regression
   • Ridge Regression ← BEST ML MODEL
   • Lasso Regression  
   • Random Forest
   • Gradient Boosting
   
   Ridge achieved: MAE 0.9431 D (30.6% improvement)
   Key insight: CCT_ratio_AL most important feature

3. PARAMETER OPTIMIZATION WITHIN SRK/T2
   Purpose: Keep formula structure, optimize internal parameters
   
   Modified parameters (clinically plausible):
   • nc (corneal refractive index): 1.333 → optimized
   • k_index (keratometric index): 1.3375 → optimized  
   • ACD offset: adjusted
   
   Result: MAE 1.3150 D (only 3.2% improvement)
   Conclusion: Formula structure too limiting

4. RIDGE-INSPIRED PARAMETER OPTIMIZATION
   Purpose: Use ML insights to guide parameter modification
   
   Approach: Let parameters vary with CCT based on Ridge patterns
   • nc = f(CCT, K)
   • k_index = f(CCT)
   • ACD_offset = f(CCT, AL)
   
   Result: Similar to basic parameter optimization
   Conclusion: Still constrained by formula structure

5. ADDITIVE CORRECTION APPROACH
   Purpose: Add correction term to standard SRK/T2
   
   Formula: REF = SRKT2_standard + Correction_term
   
   Correction includes:
   • CCT/AL ratio
   • CCT×AL interaction
   • CCT² term
   • CCT main effect
   
   Result: MAE 1.1967 D (12% improvement)

6. MULTIPLICATIVE CORRECTION APPROACH ← BEST PRACTICAL METHOD
   Purpose: Scale standard SRK/T2 by CCT-dependent factor
   
   Formula: REF = SRKT2_standard × (1 + m₀ + m₁×CCT_norm + m₂×(CCT/AL-26))
   
   Result: MAE 1.0218 D (24.8% improvement)
   Captures 81% of Ridge's improvement while keeping interpretability

7. EXTENDED RANGE OPTIMIZATION
   Purpose: Allow wider parameter ranges for diseased corneas
   
   Ranges:
   • nc: 1.25-1.40 (vs standard 1.32-1.35)
   • k_index: 1.25-1.45 (vs standard 1.32-1.35)
   
   Result: Similar to multiplicative correction

8. ULTRA-WIDE RANGE OPTIMIZATION  
   Purpose: Explore extreme ranges for severely edematous corneas
   
   Ranges:
   • nc: 1.20-1.50 (very wide)
   • k_index: 1.20-1.60 (very wide)
   • ACD offset: ±3.0 mm
   
   Result: Pending/Similar to best methods
"""

print(methods_summary)

print("\n" + "=" * 80)
print("KEY INSIGHTS:")
print("=" * 80)

insights = """
1. CCT IS CRITICAL
   • Standard SRK/T2 ignores CCT completely
   • Pre-DMEK corneas have high CCT due to edema
   • All successful methods incorporate CCT

2. FORMULA STRUCTURE MATTERS MORE THAN PARAMETERS
   • Optimizing within SRK/T2: only 3% improvement
   • Adding corrections outside: up to 25% improvement
   
3. MULTIPLICATIVE > ADDITIVE
   • Multiplicative scales with refraction magnitude
   • Better captures the proportional nature of optical errors

4. CLINICAL CONTEXT IS ESSENTIAL
   • Pre-DMEK: Measuring diseased corneas (edematous)
   • Post-surgery: Cornea normalizes but formula must account for pre-op state
   • Standard formulas assume healthy corneas - fails here

5. PRACTICAL VS THEORETICAL
   • Pure ML (Ridge): 30.6% improvement but "black box"
   • Multiplicative: 24.8% improvement with interpretability
   • 81% of benefit with clinical validity
"""

print(insights)

print("\n" + "=" * 80)
print("FINAL RECOMMENDATION:")
print("=" * 80)

print("""
For IOL calculation in combined phaco-DMEK surgery:

USE MULTIPLICATIVE CORRECTION:
REF = SRKT2_standard × (1 + m₀ + m₁×CCT_norm + m₂×(CCT/AL-26))

Where:
• SRKT2_standard = Original calculation
• CCT_norm = (CCT - 600) / 100
• Optimized coefficients: m₀=-0.5, m₁=-0.5, m₂=+0.11

Benefits:
✓ 25% reduction in prediction error
✓ Increases accuracy within ±1.0D from 49% to 67%
✓ Clinically interpretable
✓ Easy to implement
✓ Accounts for corneal edema via CCT
""")

In [ ]:
# FINAL MULTIPLICATIVE CORRECTION FORMULA
print("=" * 80)
print("FINAL FORMULA: SRK/T2-DMEK WITH MULTIPLICATIVE CORRECTION")
print("=" * 80)

print("\n📐 COMPLETE FORMULA:")
print("-" * 80)

print("""
REF = SRKT2_standard × (1 + m₀ + m₁×CCT_norm + m₂×CCT_ratio)

Where:
  • REF = Predicted post-operative refraction (D)
  • SRKT2_standard = Standard SRK/T2 calculation
  • CCT_norm = (CCT - 600) / 100
  • CCT_ratio = (CCT/AL - 26)
""")

print("\n🔢 OPTIMIZED COEFFICIENTS:")
print("-" * 80)
print(f"""
From optimization on 96 pre-DMEK patients:
  • m₀ = -0.5000
  • m₁ = -0.5000  
  • m₂ = +0.1104
""")

print("\n💻 PYTHON IMPLEMENTATION:")
print("-" * 80)
print("""
def calculate_SRKT2_DMEK(AL, K_avg, IOL_power, A_constant, CCT):
    # Step 1: Calculate standard SRK/T2
    ref_standard = calculate_SRKT2(AL, K_avg, IOL_power, A_constant)
    
    # Step 2: Calculate CCT-based correction factor
    cct_norm = (CCT - 600) / 100
    cct_ratio = (CCT / AL) - 26
    
    correction_factor = 1 + (-0.5000) + (-0.5000)*cct_norm + (0.1104)*cct_ratio
    
    # Step 3: Apply multiplicative correction
    ref_corrected = ref_standard * correction_factor
    
    return ref_corrected
""")

print("\n📊 PERFORMANCE METRICS:")
print("-" * 80)
print(f"""
Original SRK/T2:
  • MAE: 1.3591 D
  • Within ±1.0 D: 49.0%
  • Within ±0.5 D: 17.7%

With Multiplicative Correction:
  • MAE: 1.0218 D (-24.8% error)
  • Within ±1.0 D: 66.7% (+36% relative improvement)
  • Within ±0.5 D: ~30% (estimated)
""")

print("\n🔍 EXAMPLE CALCULATION:")
print("-" * 80)

# Example patient
example_patient = {
    'AL': 23.5,
    'K_avg': 44.0,
    'IOL_power': 20.0,
    'A_constant': 118.4,
    'CCT': 640  # Edematous cornea
}

# Calculate standard SRK/T2
ref_standard = calculate_SRKT2(
    example_patient['AL'], 
    example_patient['K_avg'],
    example_patient['IOL_power'],
    example_patient['A_constant']
)

# Calculate corrected
ref_corrected = calculate_SRKT2_multiplicative(
    example_patient['AL'],
    example_patient['K_avg'], 
    example_patient['IOL_power'],
    example_patient['A_constant'],
    example_patient['CCT'],
    m0=-0.5000, m1=-0.5000, m2=0.1104
)

cct_norm = (example_patient['CCT'] - 600) / 100
cct_ratio = (example_patient['CCT'] / example_patient['AL']) - 26
correction_factor = 1 + (-0.5000) + (-0.5000)*cct_norm + (0.1104)*cct_ratio

print(f"Example Patient:")
print(f"  AL = {example_patient['AL']} mm")
print(f"  K_avg = {example_patient['K_avg']} D")
print(f"  IOL = {example_patient['IOL_power']} D")
print(f"  CCT = {example_patient['CCT']} μm (edematous)")
print(f"")
print(f"Calculations:")
print(f"  Standard SRK/T2 = {ref_standard:.3f} D")
print(f"  CCT_norm = ({example_patient['CCT']}-600)/100 = {cct_norm:.2f}")
print(f"  CCT_ratio = ({example_patient['CCT']}/{example_patient['AL']})-26 = {cct_ratio:.3f}")
print(f"  Correction factor = {correction_factor:.3f}")
print(f"  ")
print(f"  CORRECTED REF = {ref_standard:.3f} × {correction_factor:.3f} = {ref_corrected:.3f} D")
print(f"")
print(f"  Difference = {ref_corrected - ref_standard:.3f} D")

print("\n⚠️ CLINICAL NOTES:")
print("-" * 80)
print("""
1. This formula is specifically for combined phaco-DMEK surgery
2. Designed for edematous corneas (Fuchs' dystrophy, endothelial dysfunction)
3. CCT measurement is CRITICAL - must be accurate
4. Validation on larger datasets recommended
5. Consider this when CCT > 600 μm (edematous state)
""")